In [11]:
### Comparaison 2SLS vs Weighting to estimate group LATE
### Jeremy L'Hour
### 15/10/2019


### FUNCTIONS AND PACKAGES
#install.packages("AER")
library("AER")

dgp <- function(n,tau.h=1,tau.l=-1){
	md_x = runif(n)
	Z = as.numeric(runif(n) < md_x)
	eta = rnorm(n,sd=.7)
	D = .2 + .5*Z + eta
    # Define groups
    gamma = rexp(n)
    high.effect     = quantile(gamma, .8)
    low.effect      = quantile(gamma, .2)
    high            = as.numeric(gamma>high.effect)
    low             = as.numeric(gamma<low.effect)
	Y = .6 + tau.l*low*D + tau.h*high*D + .5*(tau.l+tau.h)*(1-low-high)*D + (eta + rnorm(n))
	return(data.frame(Y=Y,
					  D=D,
					  Z=Z,
					  gamma=gamma,
					  md_x=md_x))
}

In [ ]:
### SIMULATIONS
R = 1000
tau0.l = -.3
tau0.h = .5
thres = .2
r = matrix(nrow=R, ncol=4)

set.seed(69742)
t_start = Sys.time()
for(i in 1:R){
	data <- dgp(n=5000,tau=tau0)

	high.effect     = quantile(data$gamma, 1-thres)
    low.effect      = quantile(data$gamma, thres)
    data[,"h"]      = as.numeric(data$gamma>high.effect)
    data[,"l"]      = as.numeric(data$gamma<low.effect)

	data[,"Dh"]     = data[,"D"]*data$h
  	data[,"Dl"]     = data[,"D"]*data$l
  	data[,"Dn"]     = data[,"D"]*(1-data$h-data$h)
  	data[,"Zhd"]    = (data[,"Z"]-data[,"md_x"])*data$h
  	data[,"Zld"]    = (data[,"Z"]-data[,"md_x"])*data$l
  	data[,"Znd"]    = (data[,"Z"]-data[,"md_x"])*(1-data$h-data$h)
  	data[,"Zh"]     = data[,"Z"]*data$h
  	data[,"Zl"]     = data[,"Z"]*data$l
  	data[,"Zn"]     = data[,"Z"]*(1-data$h-data$h)
    
    data[,"w"]      = 1/(data[,"md_x"]*(1-data[,"md_x"]))

	### A. Weighting
	reg.w <- ivreg(Y ~ Dh + Dl + Dn - 1 | Zhd + Zld + Znd - 1, data=data,weight=data[,"w"])

	### B. 2SLS
	reg.2sls <- ivreg(Y ~ Dh + Dl + Dn | Zh + Zl + Zn, data=data)

	### C. Computing

	### D. Saving results
	r[i,] <- c(coef(reg.w)[c("Dh","Dl")],coef(reg.2sls)[c("Dh","Dl")])
}
print(Sys.time()-t_start)

In [ ]:
### DISPLAY
### COMPUTE BIAS AND RMSE
StatDisplay = data.frame()
StatDisplay[1:6,"bias"] =  abs(apply(r-tau0,2,mean))
StatDisplay[1:6,"MSE"] = apply((r-tau0)^2,2,mean)
row.names(StatDisplay) = c("weighting.high","weighting.low","2SLS.high","2SLS.low")
print(StatDisplay)
